![人工智慧 - 自由團隊](https://raw.githubusercontent.com/chenkenanalytic/img/master/af/aifreeteam.png)


<center>Welcome to the course《Python: from business analytics to Artificial Intelligence》by AI . FREE Team.</center>
<br>
<center>歡迎大家來到 AI . FREE Team 《Python 從商業分析到人工智慧》的 AI 基礎教學 - Lesson 03 Neural Network (3-03)。 </center>
<br>

<center>(Author: Chen Ken；Date of published: 2021//；AI . FREE Team Website: https://aifreeblog.herokuapp.com/)</center>

# 類神經網路


在經過上一篇羅吉斯回歸的手刻演算法後，本篇教學，我們將帶大家走過基礎類神經網路的手刻模型；讀者們可以將羅吉斯回歸模型視作每一個類神經元，而神經網路即是將很多個神經元集結在做運算。

## STEP 0. 使用矩陣運算套件 (numpy)


In [ ]:
import numpy as np

## STEP 1. Sigmoid 函數

先 code 出 sigmoid 的機率函數

In [ ]:
def sigmoid(x):
  p = 1 / (1 + np.exp(-x))
  return p

## STEP 2. 建構神經網絡大小

定義「兩層」神經網路的神經元多寡。 (輸入層 & 隱藏層)

In [ ]:
def neurons(first_nums, second_nums):
  # 神經元數量
  in_num = first_nums
  # 一層
  hid_num = second_nums
  return (in_num, hid_num)

## STEP 3. 初始化權重

針對類神經網路的權重進行初始化

In [ ]:
def initial_weight(in_num, hid_num):

  a1 = np.zeros(shape=(in_num, hid_num), dtype="float32")
  b1 = np.zeros(shape=(1, hid_num), dtype="float32")
  a2 = np.zeros(shape=(hid_num, 1), dtype="float32")
  b2 = np.zeros(shape=(1, 1), dtype="float32")

  parameters = {"a1": a1,
                "b1": b1,
                "a2": a2,
                "b2": b2}

  return parameters

## STEP 4. 向前傳播 Forward Probagation

In [ ]:
def forward(parameters, x):

  a1 = parameters["a1"]
  b1 = parameters["b1"]
  a2 = parameters["a2"]
  b2 = parameters["b2"]

  # 進行向前傳播演算
  z1 = np.dot(x, a1) + b1
  y1 = sigmoid(z1)
  z2 = np.dot(y1, a2) + b2
  y2 = sigmoid(z2)

  # 暫存資訊，作為梯度下降資訊
  temp = {"z1": z1,
          "y1": y1,
          "z2": z2,
          "y2": y2}

  return y2, temp

## STEP 5. 計算損失函數

In [ ]:
def loss_function(y2, Y, parameters):

  # 找出資料筆數
  m = Y.shape[1]

  # 計算交叉商 cross-entropy loss
  logprobs = np.multiply(np.log(y2), Y) + np.multiply(np.log(1 - y2), 1 - Y)
  loss = -1 / m * np.sum(logprobs)
  loss = np.squeeze(loss)

  return loss

## STEP 6. 向後傳播 Backward Probagation

計算微分導數 (梯度)

In [ ]:
def backward(parameters, temp, x, Y):

  # 找出資料筆數
  m = x.shape[1]

  a1 = parameters["a1"]
  a2 = parameters['a2']
  y1 = temp["y1"]
  y2 = temp["y2"]

  # 計算第二層神經層權重梯度
  dz2 = y2 - Y
  da2 = 1 / m * np.dot(y1.T,dz2)
  db2 = 1 / m * np.sum(dz2, axis=1, keepdims = True)

  # 計算第一層神經層權重梯度
  dz1 = np.dot(dz2, a2.T) * (1 - np.power(y1, 2))
  da1 = 1 / m * np.dot(x.T, dz1)
  db1 = 1 / m * np.sum(dz1, axis=1, keepdims = True)

  diffs = {"da1": da1,
           "db1": db1,
           "da2": da2,
           "db2": db2}

  return diffs

## STEP 7. 透過微分導數 (diffs) 更新模型權重

In [ ]:
def updates(parameters, diffs, learning_rate = 1.0):

  a1 = parameters["a1"]
  b1 = parameters["b1"]
  a2 = parameters["a2"]
  b2 = parameters["b2"]

  da1 = diffs["da1"]
  db1 = diffs["db1"]
  da2 = diffs["da2"]
  db2 = diffs["db2"]

  # 根據梯度與學習率，更新權重
  a1 = a1 - learning_rate * da1
  b1 = b1 - learning_rate * db1
  a2 = a2 - learning_rate * da2
  b2 = b2 - learning_rate * db2

  parameters = {"a1": a1,
                "b1": b1,
                "a2": a2,
                "b2": b2}

  return parameters

## STEP 8. 合併所有自定義函數

In [ ]:
# GRADED FUNCTION: nn_model

def neural_net(x, Y, second_nums, loops = 100):

  np.random.seed(7)

  # 找出第一層的特徵值
  first_nums = x.shape[1]

  # 定義每層神經元數量 (輸入層 & 隱藏層)
  in_num, hid_num = neurons(first_nums, second_nums)

  # 初始化神經元權重
  parameters = initial_weight(in_num, hid_num)

  a1 = parameters["a1"]
  b1 = parameters["b1"]
  a2 = parameters["a2"]
  b2 = parameters["b2"]

  for i in range(0, loops):

    # 向前傳播
    y2, temp = forward(parameters, x)

    # 計算損失函數
    loss = loss_function(y2, Y, parameters)

    # 計算梯度
    diffs = backward(parameters, temp, x, Y)

    # 更新模型權重
    parameters = updates(parameters, diffs, learning_rate = 1.0)

    if i % 10 == 0:
      print ("第 ", i, " 個迴圈; 目前模型運算的成本為: ", loss)
  return parameters

## STEP 8. 運行與檢視

### (1) 建立資料

以 Lesson 1， and, or, xor 的資料集為例做學習判斷

In [ ]:
x = np.array([[0,0],[0,1],[1,0],[1,1]])

y_and = np.array([[0],[0],[0],[1]])
y_or = np.array([[0],[1],[1],[1]])
y_xor = np.array([[1],[0],[0],[1]])

print(x.shape[1])

### (2) 開始訓練模型

In [ ]:
print("【 and 資料學習】：")
parameters_and = neural_net(x, y_and, second_nums = 3)
print("----------------------------------------------------")
print("【 or 資料學習】：")
parameters_or = neural_net(x, y_or, second_nums = 3)
print("----------------------------------------------------")
print("【 xor 資料學習】：")
parameters_xor = neural_net(x, y_xor, second_nums = 3)

### (3) 預測模型

In [ ]:
def predict(parameters, x):

  y2, temp = forward(parameters, x)
  predictions = np.round_(y2)

  return predictions

### (4) 查看模型預測結果

In [ ]:
prediction_and = predict(parameters_and ,x)
print("y_and 類神經模型模型判斷正確率為: {} %".format(100-np.mean(np.abs(prediction_and-y_and))*100))

prediction_or = predict(parameters_or ,x)
print("y_or 類神經模型模型判斷正確率為: {} %".format(100-np.mean(np.abs(prediction_or-y_or))*100))

prediction_xor = predict(parameters_xor ,x)
print("y_xor 類神經模型模型判斷正確率為: {} %".format(100-np.mean(np.abs(prediction_xor-y_xor))*100))

#結論與發現

透過兩層類神經模型的手刻程式碼，可以觀察到羅吉斯回歸就好比單一個神經元，

多個羅吉斯回歸模型的堆疊，建置出一個類神經模型，同採用梯度下降(微分)來進行模型參數的更新，

從實驗結果，我們可以發現類神經模型在邏輯運算(and, or, xor)皆能夠達到 100% 判斷正確率！

成功證明類神經模型，解決複雜問題的有效性！

但...如果根據每次的任務，都需要重新撰寫程式碼，真的太過耗時，

因此科學家們推出了實用的深度學習框架，供 AI 工程師得以快速開發類神經模型！

下一篇教學，將引導讀者學習 AI 模型開源框架：Keras。

# 更深入的教學與專案實作

如果你/妳對於深入開發 AI 技術的專案有興趣，想嘗試更多、更有趣、更扎實的實務AI技術專案，

歡迎參考我們從0到1的AI技術課程：[《學習 AI 一把抓：點亮人工智慧技能樹》](https://hahow.in/cr/slashie-ai-free-team)！

# 課程文件

### AI Foundations 課程清單
- <a href="https://colab.research.google.com/github/AI-FREE-Team/Python-Basics/blob/master/documents/Lesson00%20Preface.ipynb">Lesson 00 Preface 課程前言</a>
- <a href="https://colab.research.google.com/github/AI-FREE-Team/Python-Basics/blob/master/documents/Lesson00%20Preface.ipynb">Lesson 01 MP model</a>
- <a href="https://colab.research.google.com/github/AI-FREE-Team/Python-Basics/blob/master/documents/Lesson00%20Preface.ipynb">Lesson 02 Logistic Regression</a>
- <a href="https://colab.research.google.com/github/AI-FREE-Team/Python-Basics/blob/master/documents/Lesson00%20Preface.ipynb">Lesson 03 Neural Network</a> (We are here now! --本篇課程--)
- <a href="https://colab.research.google.com/github/AI-FREE-Team/Python-Basics/blob/master/documents/Lesson00%20Preface.ipynb">Lesson 04 Deep Neural Network</a>
- <a href="https://colab.research.google.com/github/AI-FREE-Team/Python-Basics/blob/master/documents/Lesson00%20Preface.ipynb">Lesson 05 Convolution Neural Network</a>
- <a href="https://colab.research.google.com/github/AI-FREE-Team/Python-Basics/blob/master/documents/Lesson00%20Preface.ipynb">Lesson 06 Handwriting Traditional Chinese Characters Recognition</a>
- <a href="https://colab.research.google.com/github/AI-FREE-Team/Python-Basics/blob/master/documents/Lesson00%20Preface.ipynb">Lesson 07 Sentiment Analysis</a>